In [ ]:
!pip install autotrain-advanced

In [ ]:
%shell

export HF_USERNAME=your_hugging_face_username
export HF_TOKEN=your_hugging_face_write_token

In [ ]:
!git clone https://github.com/dnth/vl-hf-annotation-converter
!cd vl-hf-annotation-converter && autotrain --config configs/detr-resnet-50.yml